In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import xgboost as xgb
from classifiers import Classifiers
from nltk.corpus import stopwords
from utils import create_pipe

In [2]:
path="../data/combined.pickle"

try:
    with open(path,'rb') as handle:
        pickleload=pickle.load(handle)
except FileNotFoundError as e:
    e.strerror = "Pls run 01_scrape_reddit first to pull the data and 02_EDA to merge data."
    raise e

df=pd.DataFrame(pickleload)

df

,post,label,post2
0,this is the nail in the coffin for the idea of...,0,this is the nail in the coffin for the idea of...
1,"i’m closeted, always been, and always will be....",0,"i’m closeted, always been, and always will be...."
2,the fifa world cup in qatar should be a remind...,0,the fifa world cup in qatar should be a remind...
3,we moved from the dc metro area last year to t...,0,we moved from the dc metro area last year to t...
4,they spend so much time focusing on arbitrary ...,0,they spend so much time focusing on arbitrary ...
...,...,...,...
10078,"if jesus died for our sins, what's keeping us ...",1,"if jesus died for our sins, what's keeping u f..."
10079,hello everybody it may seem like a dumb questi...,1,hello everybody it may seem like a dumb questi...
10080,today's readings: 1 corinthians 1:4-8 &gt;i ...,1,today's readings: 1 corinthian 1:4-8 &gt;i tha...
10081,i don't propose this question in the sense of ...,1,i don't propose this question in the sense of ...


### Prepare the dataset

In [3]:
X=df['post']
y=df['label']

In [4]:
c={
        'cls':RandomForestClassifier(min_samples_split= 7, min_samples_leaf= 4, max_depth= 60),
        'name':'RandomForest',
        'tvec_params':{'min_df': 0.05, 'max_features': 3500, 'max_df': 0.8,'use_idf': False, 'ngram_range': (1, 2)},
    }

In [5]:
pipe=create_pipe(c['cls'],tfidf=True,**c['tvec_params'])

In [6]:
pipe.fit(X,y)

Pipeline(steps=[('tvec',
                 TfidfVectorizer(max_df=0.8, max_features=3500, min_df=0.05,
                                 ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 token_pattern='[^\\W\\d_]+', use_idf=False)),
                ('cls',
                 RandomForestClassifier(max_depth=60, min_samples_leaf=4,
                                        min_samples_split=7))])

In [23]:
df_test=pd.DataFrame(["""
Girlfriend of 3 years just gave me an ultimatum
For context, when we first started dating she was in a stage in her life where she was fairly ‘loose’ with her faith (Christianity). Early on, we both acknowledged and accepted each other’s beliefs, or my lack of, and I thought that was the end of it. I mean couples don’t always share the same religious beliefs right?
"""])
# df_test[0]

In [24]:
result=pipe.predict(df_test[0])
if result[0]==1:
    print("is christian")
else:
    print("is aethiest")        

is aethiest


And that is correct!